In [1]:
import pandas as pd
import numpy as np

In [2]:
# On charge nos jeux de données
df_appli_test = pd.read_csv('application_test.csv')
df = pd.read_csv('application_train.csv')
bureau = pd.read_csv('bureau.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')
prev_app = pd.read_csv('previous_application.csv')

In [3]:
# On veut tous nos clients
df = df.append(df_appli_test).reset_index()

C:\Users\victo\AppData\Local\Temp\ipykernel_30128\3370372152.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_appli_test).reset_index()


## Informations sur les clients

In [4]:
# On crée des copies pour ne pas changer nos dataframes initiaux
df_bis = df.copy()
bureau_bis = bureau.copy()

In [5]:
# On ne garde que quelques colonnes
df_bis = df_bis[['SK_ID_CURR', 'CODE_GENDER', 'DAYS_BIRTH', 'OCCUPATION_TYPE',
                 'NAME_INCOME_TYPE', 'AMT_INCOME_TOTAL', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']]

bureau_bis = bureau_bis[['SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT']]

In [6]:
# On joint les dataframes
infos_clients = df_bis.join(bureau_bis.set_index('SK_ID_CURR'), how='left', on='SK_ID_CURR')

In [7]:
# On regarde nos valeurs manquantes
infos_clients.isnull().sum()

SK_ID_CURR                  0
CODE_GENDER                 0
DAYS_BIRTH                  0
OCCUPATION_TYPE        552858
NAME_INCOME_TYPE            0
AMT_INCOME_TOTAL            0
FLAG_OWN_CAR                0
FLAG_OWN_REALTY             0
AMT_CREDIT_SUM_DEBT    308113
dtype: int64

In [8]:
# On les remplit
infos_clients['OCCUPATION_TYPE'].fillna('Not applicable / Not specified', inplace=True)

# On considère qu'ils n'avaient pas de crédit auparavant
infos_clients['AMT_CREDIT_SUM_DEBT'].fillna(0, inplace=True)

In [9]:
# On regroupe selon les numéros de prêt
infos_clients.set_index('SK_ID_CURR', inplace=True)
infos_clients['AMT_CREDIT_SUM_DEBT'] = infos_clients.groupby(['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].max()
infos_clients = infos_clients.reset_index()

In [10]:
# On remplace nos -1 par un message
infos_clients.loc[infos_clients['AMT_CREDIT_SUM_DEBT'] == -1, 'AMT_CREDIT_SUM_DEBT'] = 'Not applicable / Not specified'

In [11]:
# On supprime les doublons
print(infos_clients.shape, '\n')
infos_clients = infos_clients.drop_duplicates()
print(infos_clients.shape)

(1766872, 9) 

(356255, 9)


In [12]:
# On gère notre colonne date de naissance
infos_clients['DAYS_BIRTH'] = round(infos_clients['DAYS_BIRTH'] / 365.25 * (-1)).astype(int)

In [13]:
infos_clients['AMT_CREDIT_SUM_DEBT'] = infos_clients['AMT_CREDIT_SUM_DEBT'].astype(float)

In [14]:
# On renommne nos colonnes
infos_clients.columns = ['ID_PRET', 'SEXE', 'AGE', 'OCCUPATION', 'SOURCE_REVENU', 'REVENU_TOTAL', 'VOITURE', 'IMMOBILIER', 'DETTE_TOTALE']

In [15]:
# On vérifie
infos_clients.head()

,ID_PRET,SEXE,AGE,OCCUPATION,SOURCE_REVENU,REVENU_TOTAL,VOITURE,IMMOBILIER,DETTE_TOTALE
0,100002,M,26,Laborers,Working,202500.0,N,Y,245781.0
8,100003,F,46,Core staff,State servant,270000.0,N,N,0.0
12,100004,M,52,Laborers,Working,67500.0,Y,Y,0.0
14,100006,F,52,Laborers,Working,135000.0,N,Y,0.0
15,100007,M,55,Core staff,Working,121500.0,N,Y,0.0


In [16]:
df_sample = pd.read_csv('data_preprocessed_sample.csv')

In [17]:
df_sample.head()

,Unnamed: 0,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_NAME_CONTRACT_STATUS_Refused_SUM,CC_NAME_CONTRACT_STATUS_Sent proposal_MIN,CC_NAME_CONTRACT_STATUS_Sent proposal_MAX,CC_NAME_CONTRACT_STATUS_Sent proposal_MEAN,CC_NAME_CONTRACT_STATUS_Sent proposal_SUM,CC_NAME_CONTRACT_STATUS_Signed_MIN,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_COUNT
0,271920,271920,415254,0.0,0,0,0,90000.0,178290.0,10084.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,59440,59440,168900,0.0,1,0,0,90000.0,191880.0,18688.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,164858,164858,291107,0.0,1,0,0,180000.0,405000.0,20250.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,64434,64434,174719,0.0,0,1,1,292500.0,1506816.0,47443.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0
4,216735,216735,351129,0.0,1,0,1,112500.0,247275.0,19953.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
loan_ids = df_sample['SK_ID_CURR'].unique().tolist()

In [19]:
infos_clients = infos_clients[infos_clients['ID_PRET'].isin(loan_ids)]

In [20]:
infos_clients.shape

(3563, 9)

In [21]:
# On sauvegarde
df_csv_clients = infos_clients.to_csv('infos_clients.csv')

## Informations sur les prêts

In [22]:
# On crée des copies pour ne pas changer nos dataframes initiaux
df_bis = df.copy()
prev_app_bis = prev_app.copy()
bureau_bis = bureau.copy()
bureau_balance_bis = bureau_balance.copy()

In [23]:
# On ne garde que quelques colonnes
df_bis = df_bis[['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_CREDIT', 'AMT_ANNUITY',
                 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']]

prev_app_bis = prev_app_bis[['SK_ID_CURR', 'CODE_REJECT_REASON', 'NAME_CASH_LOAN_PURPOSE']]

bureau_bis = bureau_bis[['SK_ID_CURR', 'SK_ID_BUREAU', 'AMT_CREDIT_MAX_OVERDUE', 'AMT_CREDIT_SUM_OVERDUE']]

bureau_balance_bis = bureau_balance_bis[['SK_ID_BUREAU', 'STATUS']]

In [24]:
# On joint les dataframes bureau et bureau_balance
bureau_bis = bureau_bis.join(bureau_balance_bis.set_index('SK_ID_BUREAU'), how='left', on='SK_ID_BUREAU')

In [25]:
# On regarde les valeurs manquantes
bureau_bis.isnull().sum()

SK_ID_CURR                       0
SK_ID_BUREAU                     0
AMT_CREDIT_MAX_OVERDUE    18130741
AMT_CREDIT_SUM_OVERDUE           0
STATUS                      942074
dtype: int64

In [26]:
# On remplit les valeurs manquantes
bureau_bis['STATUS'].fillna('Not applicable / Not specified', inplace=True)
bureau_bis['AMT_CREDIT_MAX_OVERDUE'].fillna(-1, inplace=True)

In [27]:
# On regroupe selon les numéros de prêt
bureau_bis['AMT_CREDIT_MAX_OVERDUE'] = bureau_bis.groupby(['SK_ID_CURR'])['AMT_CREDIT_MAX_OVERDUE'].max()
bureau_bis['AMT_CREDIT_SUM_OVERDUE'] = bureau_bis.groupby(['SK_ID_CURR'])['AMT_CREDIT_SUM_OVERDUE'].max()

In [28]:
# On regarde les valeurs manquantes
bureau_bis.isnull().sum()

SK_ID_CURR                       0
SK_ID_BUREAU                     0
AMT_CREDIT_MAX_OVERDUE    19672252
AMT_CREDIT_SUM_OVERDUE    19672252
STATUS                           0
dtype: int64

In [29]:
# On remplit les valeurs manquantes
bureau_bis.fillna(0, inplace=True)

In [30]:
# On supprime les doublons
print(bureau_bis.shape, '\n')
bureau_bis = bureau_bis.drop_duplicates()
print(bureau_bis.shape)

(25121815, 5) 

(2573172, 5)


In [31]:
# On joint les dataframes
infos_prets = df_bis.join(prev_app_bis.set_index('SK_ID_CURR'), how='left', on='SK_ID_CURR')
infos_prets = infos_prets.join(bureau_bis.set_index('SK_ID_CURR'), how='left', on='SK_ID_CURR')
infos_prets.drop(columns=['SK_ID_BUREAU'], inplace=True)

In [32]:
# On regarde les valeurs manquantes
infos_prets.isnull().sum()

SK_ID_CURR                      0
NAME_CONTRACT_TYPE              0
AMT_CREDIT                      0
AMT_ANNUITY                  1876
EXT_SOURCE_1              7327887
EXT_SOURCE_2                 8572
EXT_SOURCE_3               526619
CODE_REJECT_REASON          90218
NAME_CASH_LOAN_PURPOSE      90218
AMT_CREDIT_MAX_OVERDUE     203367
AMT_CREDIT_SUM_OVERDUE     203367
STATUS                     203367
dtype: int64

In [33]:
# On arrondit quelques valeurs
for col in ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']:
    infos_prets[col] = infos_prets[col].round(2)

In [34]:
# On remplit les valeurs manquantes
infos_prets.fillna(0, inplace=True)

In [35]:
# On supprime les doublons
print(infos_prets.shape, '\n')
infos_prets = infos_prets.drop_duplicates()
print(infos_prets.shape)

(14710209, 12) 

(1876389, 12)


In [36]:
# On regarde les premières lignes
infos_prets.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_CREDIT,AMT_ANNUITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,CODE_REJECT_REASON,NAME_CASH_LOAN_PURPOSE,AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_SUM_OVERDUE,STATUS
0,100002,Cash loans,406597.5,24700.5,0.08,0.26,0.14,XAP,XAP,0.0,0.0,C
0,100002,Cash loans,406597.5,24700.5,0.08,0.26,0.14,XAP,XAP,0.0,0.0,0
0,100002,Cash loans,406597.5,24700.5,0.08,0.26,0.14,XAP,XAP,0.0,0.0,1
0,100002,Cash loans,406597.5,24700.5,0.08,0.26,0.14,XAP,XAP,0.0,0.0,X
1,100003,Cash loans,1293502.5,35698.5,0.31,0.62,0.00,XAP,XNA,0.0,0.0,Not applicable / Not specified


In [37]:
# On ne garde qu'une seule ligne par prêt
infos_prets = infos_prets.drop_duplicates(subset='SK_ID_CURR')

In [38]:
# On renommne nos colonnes
infos_prets.columns = ['ID_PRET', 'TYPE_CREDIT', 'MONTANT_CREDIT', 'MONTANT_ANNUITE', 'SCORE_EXT_1', 'SCORE_EXT_2', 'SCORE_EXT_3', 'CODE_REJET', 'OBJET_PRET', 'RETARD_PAIEMENT_MAX', 'RETARD_PAIEMENT_TOTAL', 'STATUT_PRET']

In [39]:
# On vérifie
infos_prets.head()

,ID_PRET,TYPE_CREDIT,MONTANT_CREDIT,MONTANT_ANNUITE,SCORE_EXT_1,SCORE_EXT_2,SCORE_EXT_3,CODE_REJET,OBJET_PRET,RETARD_PAIEMENT_MAX,RETARD_PAIEMENT_TOTAL,STATUT_PRET
0,100002,Cash loans,406597.5,24700.5,0.08,0.26,0.14,XAP,XAP,0.000,0.0,C
1,100003,Cash loans,1293502.5,35698.5,0.31,0.62,0.00,XAP,XNA,0.000,0.0,Not applicable / Not specified
2,100004,Revolving loans,135000.0,6750.0,0.00,0.56,0.73,XAP,XAP,10172.205,0.0,Not applicable / Not specified
3,100006,Cash loans,312682.5,29686.5,0.00,0.65,0.00,XAP,XNA,0.000,0.0,0
4,100007,Cash loans,513000.0,21865.5,0.00,0.32,0.00,XAP,XNA,0.000,0.0,Not applicable / Not specified


In [40]:
# On regarde les dimensions de notre dataframe
infos_prets.shape

(356255, 12)

In [41]:
infos_prets = infos_prets[infos_prets['ID_PRET'].isin(loan_ids)]

In [42]:
# On sauvegarde
df_csv_prets = infos_prets.to_csv('infos_prets.csv')